# Speed up g(r) code

I want to compare the timings for calculating the distances between many atoms.  This involves computing the double loop over all coordinates.

In [55]:
import time
import os.path as op

In [56]:
from sasmol import sasmol

In [57]:
import numpy as np

In [58]:
from scipy.spatial.distance import pdist

In [59]:
import concurrent.futures
import multiprocessing

n_cpu = multiprocessing.cpu_count()
print('n_cpu = {}'.format(n_cpu))

n_cpu = 4


In [52]:
import sys
sys.path.append('./')
from distance import distance

In [5]:
# import dask.array as da # NOT APPROPRIATE FOR PROBLEM

In [6]:
# setup inputs
pdb_fname = 'test.pdb'
dcd_fname = 'test.dcd'
mol = sasmol.SasMol(0)
mol.read_pdb(pdb_fname)

reading filename:  test.pdb
num_atoms =  2048
>>> found  1  model(s) or frame(s)
finished reading frame =  1


In [7]:
# confirm indexing
n = 3
d = np.zeros([n, n])
for j in xrange(3):
    for k in xrange(j+1, 3):
        d[j, k] = 1
print(d)

[[ 0.  1.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]]


In [8]:
# confirm products
c1 = np.arange(3)
c2 = np.arange(3)/10.0
dsum = np.sqrt((c1[0] - c2[0]) ** 2  + (c1[1] - c2[1]) ** 2  + (c1[2] - c2[2]) ** 2)
nsum = np.sqrt(np.sum((c1 - c2) ** 2))
print(dsum, nsum)

(2.0124611797498111, 2.0124611797498111)


In [36]:
%%timeit
# ignorant python
dcd_file = mol.open_dcd_read(dcd_fname)

n_atoms = mol.natoms()
n_frames = dcd_file[2]

print('number of frames: {}'.format(n_frames))
print('number of atoms: {}'.format(n_atoms))

dist = np.zeros([n_frames, n_atoms, n_atoms])
for i in xrange(n_frames):
    mol.read_dcd_step(dcd_file, i)
    coor = mol.coor()[0]
    for j in xrange(n_atoms):
        for k in xrange(j+1, n_atoms):
            dist[i, j, k] = np.sqrt(np.sum((coor[j] - coor[k]) ** 2))
            
mol.close_dcd_read(dcd_file[0])

number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
1 loop, best of 3: 1min per loop


In [9]:
# ignorant python
dcd_file = mol.open_dcd_read(dcd_fname)

n_atoms = mol.natoms()
n_frames = dcd_file[2]

print('number of frames: {}'.format(n_frames))
print('number of atoms: {}'.format(n_atoms))

dist = np.zeros([n_frames, n_atoms, n_atoms])
for i in xrange(n_frames):
    mol.read_dcd_step(dcd_file, i)
    coor = mol.coor()[0]
    for j in xrange(n_atoms):
        for k in xrange(j+1, n_atoms):
            dist[i, j, k] = np.sqrt(np.sum((coor[j] - coor[k]) ** 2))
            
mol.close_dcd_read(dcd_file[0])

dp = dist[-1, :4, :4]

number of frames: 1
number of atoms: 2048
.result =  0


In [10]:
dp

array([[  0.        ,   9.51893197,  12.39790455,  10.83006633],
       [  0.        ,   0.        ,   7.84797516,   4.79180904],
       [  0.        ,   0.        ,   0.        ,  12.50089329],
       [  0.        ,   0.        ,   0.        ,   0.        ]])

In [56]:
%%timeit
# numpy version
dcd_file = mol.open_dcd_read(dcd_fname)

n_atoms = mol.natoms()
n_frames = dcd_file[2]

print('number of frames: {}'.format(n_frames))
print('number of atoms: {}'.format(n_atoms))

dist = np.zeros([n_frames, n_atoms, n_atoms])
for i in xrange(n_frames):
    mol.read_dcd_step(dcd_file, i)
    coor = mol.coor()[0]
    dist[i] = np.sqrt(((coor[:, None, :] - coor) ** 2).sum(-1))
    
mol.close_dcd_read(dcd_file[0])

number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
1 loop, best of 3: 1.11 s per loop


In [57]:
# numpy version
dcd_file = mol.open_dcd_read(dcd_fname)

n_atoms = mol.natoms()
n_frames = dcd_file[2]

print('number of frames: {}'.format(n_frames))
print('number of atoms: {}'.format(n_atoms))

dist = np.zeros([n_frames, n_atoms, n_atoms])
for i in xrange(n_frames):
    mol.read_dcd_step(dcd_file, i)
    coor = mol.coor()[0]
    dist[i] = np.sqrt(((coor[:, None, :] - coor) ** 2).sum(-1))
    
mol.close_dcd_read(dcd_file[0])

dn = dist[-1, :4, :4]

number of frames: 5
number of atoms: 2048
.....result =  0


In [13]:
dp, dn

(array([[  0.        ,   9.51893197,  12.39790455,  10.83006633],
        [  0.        ,   0.        ,   7.84797516,   4.79180904],
        [  0.        ,   0.        ,   0.        ,  12.50089329],
        [  0.        ,   0.        ,   0.        ,   0.        ]]),
 array([[  0.        ,   9.51893197,  12.39790455,  10.83006633],
        [  9.51893197,   0.        ,   7.84797516,   4.79180904],
        [ 12.39790455,   7.84797516,   0.        ,  12.50089329],
        [ 10.83006633,   4.79180904,  12.50089329,   0.        ]]))

In [14]:
print(dp-dn)

[[  0.           0.           0.           0.        ]
 [ -9.51893197   0.           0.           0.        ]
 [-12.39790455  -7.84797516   0.           0.        ]
 [-10.83006633  -4.79180904 -12.50089329   0.        ]]


Even though the NumPy version calculates the entire matrix, it is more than 2x faster.

In [58]:
%%timeit
# scipy version 1
dcd_file = mol.open_dcd_read(dcd_fname)

n_atoms = mol.natoms()
n_frames = dcd_file[2]

print('number of frames: {}'.format(n_frames))
print('number of atoms: {}'.format(n_atoms))

dist = np.zeros([n_frames, n_atoms, n_atoms])

for i in xrange(n_frames):
    mol.read_dcd_step(dcd_file, i)
    coor = mol.coor()[0]
    row, col = np.triu_indices(n_atoms, 1)
    dist[i, row, col] = pdist(coor)
    
mol.close_dcd_read(dcd_file[0])

number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
1 loop, best of 3: 367 ms per loop


In [59]:
# scipy version 1
dcd_file = mol.open_dcd_read(dcd_fname)

n_atoms = mol.natoms()
n_frames = dcd_file[2]

print('number of frames: {}'.format(n_frames))
print('number of atoms: {}'.format(n_atoms))

dist = np.zeros([n_frames, n_atoms, n_atoms])

for i in xrange(n_frames):
    mol.read_dcd_step(dcd_file, i)
    coor = mol.coor()[0]
    row, col = np.triu_indices(n_atoms, 1)
    dist[i, row, col] = pdist(coor)
    
mol.close_dcd_read(dcd_file[0])

ds1 = dist[-1, :4, :4]

number of frames: 5
number of atoms: 2048
.....result =  0


In [60]:
%%timeit
# scipy version 2 (with boolean-indexing)
dcd_file = mol.open_dcd_read(dcd_fname)

n_atoms = mol.natoms()
n_frames = dcd_file[2]

print('number of frames: {}'.format(n_frames))
print('number of atoms: {}'.format(n_atoms))

dist = np.zeros([n_frames, n_atoms, n_atoms])
r = np.arange(n_atoms)

for i in xrange(n_frames):
    mol.read_dcd_step(dcd_file, i)
    coor = mol.coor()[0]
    dist[i, r[:, None] < r] = pdist(coor)
    
mol.close_dcd_read(dcd_file[0])

number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
1 loop, best of 3: 311 ms per loop


In [61]:
# scipy version 2 (with boolean-indexing)
dcd_file = mol.open_dcd_read(dcd_fname)

n_atoms = mol.natoms()
n_frames = dcd_file[2]

print('number of frames: {}'.format(n_frames))
print('number of atoms: {}'.format(n_atoms))

dist = np.zeros([n_frames, n_atoms, n_atoms])
r = np.arange(n_atoms)

for i in xrange(n_frames):
    mol.read_dcd_step(dcd_file, i)
    coor = mol.coor()[0]
    dist[i, r[:, None] < r] = pdist(coor)
    
mol.close_dcd_read(dcd_file[0])

ds2 = dist[-1, :4, :4]

number of frames: 5
number of atoms: 2048
.....result =  0


In [68]:
dp, dn, ds1, ds2

(array([[  0.        ,   9.51893197,  12.39790455,  10.83006633],
        [  0.        ,   0.        ,   7.84797516,   4.79180904],
        [  0.        ,   0.        ,   0.        ,  12.50089329],
        [  0.        ,   0.        ,   0.        ,   0.        ]]),
 array([[  0.        ,   9.55838011,  12.49256537,  11.12902151],
        [  9.55838011,   0.        ,   7.85849646,   4.99603282],
        [ 12.49256537,   7.85849646,   0.        ,  12.75118492],
        [ 11.12902151,   4.99603282,  12.75118492,   0.        ]]),
 array([[  0.        ,   9.55838011,  12.49256537,  11.12902151],
        [  0.        ,   0.        ,   7.85849646,   4.99603282],
        [  0.        ,   0.        ,   0.        ,  12.75118492],
        [  0.        ,   0.        ,   0.        ,   0.        ]]),
 array([[  0.        ,   9.55838011,  12.49256537,  11.12902151],
        [  0.        ,   0.        ,   7.85849646,   4.99603282],
        [  0.        ,   0.        ,   0.        ,  12.75118492],
    

In [71]:
print(dn-ds1)
print(dn-ds2)

[[  0.           0.           0.           0.        ]
 [  9.55838011   0.           0.           0.        ]
 [ 12.49256537   7.85849646   0.           0.        ]
 [ 11.12902151   4.99603282  12.75118492   0.        ]]
[[  0.           0.           0.           0.        ]
 [  9.55838011   0.           0.           0.        ]
 [ 12.49256537   7.85849646   0.           0.        ]
 [ 11.12902151   4.99603282  12.75118492   0.        ]]


In [65]:
%%timeit
# fortran version
dcd_file = mol.open_dcd_read(dcd_fname)

n_atoms = mol.natoms()
n_frames = dcd_file[2]

print('number of frames: {}'.format(n_frames))
print('number of atoms: {}'.format(n_atoms))

dist = np.zeros([n_frames, n_atoms, n_atoms])

for i in xrange(n_frames):
    mol.read_dcd_step(dcd_file, i)
    coor = mol.coor()[0]
    dist[0] = distance(coor, dist[0])
    
mol.close_dcd_read(dcd_file[0])

number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
1 loop, best of 3: 717 ms per loop


In [75]:
# fortran version
dcd_file = mol.open_dcd_read(dcd_fname)

n_atoms = mol.natoms()
n_frames = dcd_file[2]

print('number of frames: {}'.format(n_frames))
print('number of atoms: {}'.format(n_atoms))

dist = np.zeros([n_frames, n_atoms, n_atoms])

for i in xrange(n_frames):
    mol.read_dcd_step(dcd_file, i)
    coor = mol.coor()[0]
    dist[i] = distance(coor, dist[0])
    
mol.close_dcd_read(dcd_file[0])

df = dist[-1, :4, :4]

number of frames: 5
number of atoms: 2048
.....result =  0


In [76]:
dn, df

(array([[  0.        ,   9.55838011,  12.49256537,  11.12902151],
        [  9.55838011,   0.        ,   7.85849646,   4.99603282],
        [ 12.49256537,   7.85849646,   0.        ,  12.75118492],
        [ 11.12902151,   4.99603282,  12.75118492,   0.        ]]),
 array([[  0.        ,   9.55838011,  12.49256537,  11.12902151],
        [  0.        ,   0.        ,   7.85849646,   4.99603282],
        [  0.        ,   0.        ,   0.        ,  12.75118492],
        [  0.        ,   0.        ,   0.        ,   0.        ]]))

In [78]:
dn - df

array([[  0.        ,   0.        ,   0.        ,   0.        ],
       [  9.55838011,   0.        ,   0.        ,   0.        ],
       [ 12.49256537,   7.85849646,   0.        ,   0.        ],
       [ 11.12902151,   4.99603282,  12.75118492,   0.        ]])

# Now with [Concurrent.futures](https://docs.python.org/3.2/library/concurrent.futures.html)

In [80]:
def get_dist(coor, dist, r):
    dist[r[:, None] < r] = pdist(coor)

In [100]:
%%timeit
# ThreadPoolExecutor.submit
dcd_file = mol.open_dcd_read(dcd_fname)

n_atoms = mol.natoms()
n_frames = dcd_file[2]

print('number of frames: {}'.format(n_frames))
print('number of atoms: {}'.format(n_atoms))

r = np.arange(n_atoms)
dist = np.zeros([n_frames, n_atoms, n_atoms])
with concurrent.futures.ThreadPoolExecutor(max_workers=n_cpu) as executor:
    for i in xrange(n_frames):
        mol.read_dcd_step(dcd_file, i)
        coor = mol.coor()[0]
        executor.submit(get_dist, coor, dist[i], r)
    
mol.close_dcd_read(dcd_file[0])

number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result = 

In [111]:
%%timeit
# ThreadPoolExecutor.map
dcd_file = mol.open_dcd_read(dcd_fname)

n_atoms = mol.natoms()
n_frames = dcd_file[2]

print('number of frames: {}'.format(n_frames))
print('number of atoms: {}'.format(n_atoms))

r = np.arange(n_atoms)
dist = np.zeros([n_frames, n_atoms, n_atoms])
with concurrent.futures.ThreadPoolExecutor(max_workers=n_cpu) as executor:
    for i in xrange(n_frames):
        mol.read_dcd_step(dcd_file, i)
        coor = mol.coor()[0]
        executor.(get_dist, coor, dist[i], r)
    
mol.close_dcd_read(dcd_file[0])

number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
1 loop, best of 3: 936 ms per loop


In [92]:
%%timeit
# ProcessPoolExecutor.submit
dcd_file = mol.open_dcd_read(dcd_fname)

n_atoms = mol.natoms()
n_frames = dcd_file[2]

print('number of frames: {}'.format(n_frames))
print('number of atoms: {}'.format(n_atoms))

r = np.arange(n_atoms)
dist = np.zeros([n_frames, n_atoms, n_atoms])
with concurrent.futures.ProcessPoolExecutor(max_workers=n_cpu) as executor:
    for i in xrange(n_frames):
        mol.read_dcd_step(dcd_file, i)
        coor = mol.coor()[0]
        executor.submit(get_dist, coor, dist[i], r)
    
mol.close_dcd_read(dcd_file[0])

number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
1 loop, best of 3: 764 ms per loop


In [93]:
%%timeit
# ProcessPoolExecutor.map
dcd_file = mol.open_dcd_read(dcd_fname)

n_atoms = mol.natoms()
n_frames = dcd_file[2]

print('number of frames: {}'.format(n_frames))
print('number of atoms: {}'.format(n_atoms))

r = np.arange(n_atoms)
dist = np.zeros([n_frames, n_atoms, n_atoms])
with concurrent.futures.ProcessPoolExecutor(max_workers=n_cpu) as executor:
    for i in xrange(n_frames):
        mol.read_dcd_step(dcd_file, i)
        coor = mol.coor()[0]
        executor.map(get_dist, coor, dist[i], r)
    
mol.close_dcd_read(dcd_file[0])

number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.

Traceback (most recent call last):
  File "/Users/schowell/data/myPrograms/anaconda3/envs/sassie/lib/python2.7/multiprocessing/queues.py", line 268, in _feed
    send(obj)
IOError: [Errno 32] Broken pipe


....result =  0
number of frames: 5
number of atoms: 2048
.....result =  0
number of frames: 5
number of atoms: 2048
.

Traceback (most recent call last):
  File "/Users/schowell/data/myPrograms/anaconda3/envs/sassie/lib/python2.7/multiprocessing/queues.py", line 268, in _feed
    send(obj)
IOError: [Errno 32] Broken pipe


....result =  0
1 loop, best of 3: 2.23 s per loop


`ThreadPoolExecutor` with `submit()` is much faster, probably because I did not setup iterators for `map()` (not sure about `ProcessPoolExecutor`)

In [28]:
# ThreadPoolExecutor with submit
dcd_file = mol.open_dcd_read(dcd_fname)

n_atoms = mol.natoms()
n_frames = dcd_file[2]

print('number of frames: {}'.format(n_frames))
print('number of atoms: {}'.format(n_atoms))

r = np.arange(n_atoms)
dist = np.zeros([n_frames, n_atoms, n_atoms])
with concurrent.futures.ThreadPoolExecutor(max_workers=n_cpu) as executor:
    for i in xrange(n_frames):
        mol.read_dcd_step(dcd_file, i)
        coor = mol.coor()[0]
        executor.submit(get_dist, coor, dist[i], r)
    
mol.close_dcd_read(dcd_file[0])
dc = dist[-1, :4, :4]

number of frames: 5
number of atoms: 2048


NameError: name 'concurrent' is not defined

In [83]:
dn, dc

(array([[  0.        ,   9.55838011,  12.49256537,  11.12902151],
        [  9.55838011,   0.        ,   7.85849646,   4.99603282],
        [ 12.49256537,   7.85849646,   0.        ,  12.75118492],
        [ 11.12902151,   4.99603282,  12.75118492,   0.        ]]),
 array([[  0.        ,   9.55838011,  12.49256537,  11.12902151],
        [  0.        ,   0.        ,   7.85849646,   4.99603282],
        [  0.        ,   0.        ,   0.        ,  12.75118492],
        [  0.        ,   0.        ,   0.        ,   0.        ]]))

In [84]:
dn - dc

array([[  0.        ,   0.        ,   0.        ,   0.        ],
       [  9.55838011,   0.        ,   0.        ,   0.        ],
       [ 12.49256537,   7.85849646,   0.        ,   0.        ],
       [ 11.12902151,   4.99603282,  12.75118492,   0.        ]])

## Now with the real data set

In [46]:
# setup inputs

run_path = '../../simulations/lj_sphere_monomer/runs/p_0p14/output'
pdb_fname = 'run2.pdb'
dcd_fname = 'run2.dcd'

pdb_fname = op.join(run_path, pdb_fname)
dcd_fname = op.join(run_path, dcd_fname)

mol = sasmol.SasMol(0)
mol.read_pdb(pdb_fname)

reading filename:  ../../simulations/lj_sphere_monomer/runs/p_0p14/output/run2.pdb
num_atoms =  2048
>>> found  1  model(s) or frame(s)
finished reading frame =  1


In [53]:
def get_dist(coor, dist, r):
    dist[r[:, None] < r] = pdist(coor)

In [ ]:
dcd_file = mol.open_dcd_read(dcd_fname)

n_atoms = mol.natoms()
n_frames = dcd_file[2]

print('number of frames: {}'.format(n_frames))
print('number of atoms: {}'.format(n_atoms))

r = np.arange(n_atoms)
dist = np.zeros([n_frames, n_atoms, n_atoms])
tic = time.time()
with concurrent.futures.ThreadPoolExecutor(max_workers=n_cpu) as executor:
    for i in xrange(n_frames):
        mol.read_dcd_step(dcd_file, i)
        coor = mol.coor()[0]
        executor.submit(get_dist, coor, dist[i], r)

toc = time.time() - tic    

mol.close_dcd_read(dcd_file[0])

print toc

number of frames: 25001
number of atoms: 2048
..........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

# Now with [Dask](http://dask.pydata.org/en/latest/)

In [33]:
def get_dist(coor, dist, r):
    dist[r[:, None] < r] = pdist(coor)
    return dist

In [39]:
from dask import delayed, compute

In [41]:
slices = [coor[i] for i in range(coor.shape[0])] # create an iterator
slices2 = [delayed(pdist)(slice) for slice in slices]
results = compute(*slices2)

ValueError: A 2-dimensional array must be passed.

Traceback
---------
  File "/Users/schowell/data/myPrograms/anaconda3/envs/sassie/lib/python2.7/site-packages/dask/async.py", line 263, in execute_task
    result = _execute_task(task, data)
  File "/Users/schowell/data/myPrograms/anaconda3/envs/sassie/lib/python2.7/site-packages/dask/async.py", line 245, in _execute_task
    return func(*args2)
  File "/Users/schowell/data/myPrograms/anaconda3/envs/sassie/lib/python2.7/site-packages/scipy/spatial/distance.py", line 1217, in pdist
    raise ValueError('A 2-dimensional array must be passed.')


In [29]:
%%timeit
# Dask with scipy version 2 (with boolean-indexing)
dcd_file = mol.open_dcd_read(dcd_fname)

n_atoms = mol.natoms()
n_frames = dcd_file[2]

print('number of frames: {}'.format(n_frames))
print('number of atoms: {}'.format(n_atoms))

r = np.arange(n_atoms)
dist = np.zeros([n_frames, n_atoms, n_atoms])
for i in xrange(n_frames):
    mol.read_dcd_step(dcd_file, i)
    coor = mol.coor()[0]
    delayed(get_dist)(coor, dist[i], r)

mol.close_dcd_read(dcd_file[0])

number of frames: 5
number of atoms: 2048
.....result =  0


In [37]:
# Dask with scipy version 2 (with boolean-indexing)
dcd_file = mol.open_dcd_read(dcd_fname)

n_atoms = mol.natoms()
n_frames = dcd_file[2]

print('number of frames: {}'.format(n_frames))
print('number of atoms: {}'.format(n_atoms))

r = np.arange(n_atoms)
dist = np.zeros([n_frames, n_atoms, n_atoms])
res = []
for i in xrange(n_frames):
    mol.read_dcd_step(dcd_file, i)
    coor = mol.coor()[0]
    res.append(delayed(get_dist)(coor, dist[i], r))

mol.close_dcd_read(dcd_file[0])
test = [calc.compute for calc in res]
dd = dist[-1, :4, :4]

number of frames: 5
number of atoms: 2048
.....result =  0


In [38]:
dd

array([[ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [31]:
dist

array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       